In [ ]:
import numpy as np
import pandas as pd
from pylab import mpl, plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.set(style='whitegrid')
mpl.rcParams['font.family'] = 'serif'
%config InlineBackend.figure_format = 'svg'

# Financial Sentiment Analysis with FinBERT

**Learning Objectives:**
- Understand sentiment analysis for financial text
- Use HuggingFace Transformers for NLP tasks
- Apply FinBERT model to score financial headlines
- Build cumulative sentiment indicators for trading signals
- Analyze sentiment-price relationships

**What is Sentiment Analysis?**

Sentiment analysis extracts subjective information from text to determine whether the content expresses positive, negative, or neutral opinions. In finance, news sentiment can:
- Predict short-term price movements
- Complement technical analysis
- Identify market regime changes
- Provide early warning signals

**FinBERT Model:**

FinBERT is a pre-trained NLP model specifically fine-tuned on financial text. Unlike general sentiment models, it understands financial jargon and context (e.g., "beat earnings" is positive, "missed guidance" is negative).

---

# PART 1: HuggingFace Transformers Pipeline

**HuggingFace Transformers** is a powerful library that provides easy access to state-of-the-art NLP models.

**Pipeline API** simplifies using pre-trained models:
- Handles tokenization automatically
- Manages model loading and inference
- Returns easy-to-use results
- Supports various tasks (sentiment, classification, translation, etc.)

Let's start by installing and importing the necessary libraries.

## 1.1 Install and Import Transformers

First, we need to install the transformers library. Run this cell if you haven't installed it yet.

In [ ]:
# Install transformers library (uncomment if needed)
# !pip install transformers torch

# Import transformers
from transformers import pipeline
print("Transformers library imported successfully!")

## 1.2 Understanding the Pipeline API

The Pipeline API is the easiest way to use pre-trained models. It abstracts away all the complexity:

**Pipeline Workflow:**
1. **Input Text** → You provide raw text
2. **Tokenization** → Converts text to model-readable format
3. **Model Inference** → Runs the neural network
4. **Post-processing** → Converts output to human-readable format
5. **Results** → Returns predictions with confidence scores

Let's test with a simple sentiment analysis example using a general model first.

In [ ]:
# Create a basic sentiment analysis pipeline
# This uses a general-purpose sentiment model (not finance-specific yet)
basic_sentiment = pipeline("sentiment-analysis")

# Test with simple examples
test_texts = [
    "This is a great product!",
    "I'm very disappointed with the service.",
    "The weather is okay today."
]

print("Basic Sentiment Analysis Examples:")
print("="*60)
for text in test_texts:
    result = basic_sentiment(text)[0]
    print(f"Text: {text}")
    print(f"  → Label: {result['label']}, Score: {result['score']:.4f}\n")

---

# PART 2: FinBERT - Finance-Specific Sentiment Analysis

**Why FinBERT?**

General sentiment models often misinterpret financial language:
- "Company beats estimates" → Should be POSITIVE (not neutral)
- "Stock plunges on earnings miss" → NEGATIVE
- "Volatility increases" → Context-dependent

**FinBERT** (ProsusAI/finbert) is trained on financial news and understands:
- Financial terminology
- Market-specific context
- Earnings reports language
- Industry jargon

Let's load the FinBERT model and compare it with the general model.

## 2.1 Load FinBERT Model

FinBERT returns three sentiment classes:
- **Positive**: Bullish news
- **Negative**: Bearish news  
- **Neutral**: Factual/non-directional information

In [ ]:
# Load FinBERT model using pipeline
# This will download the model on first run (may take a minute)
finbert = pipeline(
    "sentiment-analysis",
    model="ProsusAI/finbert",
    tokenizer="ProsusAI/finbert"
)

print("FinBERT model loaded successfully!")
print("Model: ProsusAI/finbert")
print("Classes: Positive, Negative, Neutral")

## 2.2 Test FinBERT on Financial Headlines

Let's test FinBERT on real financial headlines to see how it interprets financial language.

In [ ]:
# Financial news headlines for testing
financial_headlines = [
    "Apple reports record quarterly earnings, beating analyst expectations",
    "Tesla stock plunges 15% after disappointing delivery numbers",
    "Federal Reserve holds interest rates steady at current levels",
    "Amazon announces major layoffs affecting 10,000 employees",
    "Microsoft announces $10 billion investment in AI infrastructure",
    "Oil prices surge on supply concerns",
    "Goldman Sachs upgrades tech stocks to overweight",
    "Inflation data comes in higher than expected, markets tumble",
    "Company files for bankruptcy protection amid debt crisis",
    "Tech giant completes acquisition of AI startup for undisclosed sum"
]

print("FinBERT Sentiment Analysis - Financial Headlines")
print("="*60)

In [ ]:
# Score each headline with FinBERT
results = []
for headline in financial_headlines:
    sentiment = finbert(headline)[0]
    results.append({
        'headline': headline,
        'label': sentiment['label'],
        'score': sentiment['score']
    })
    print(f"Headline: {headline[:60]}...")
    print(f"  → Sentiment: {sentiment['label']:8s} (confidence: {sentiment['score']:.4f})\n")

# Create DataFrame for easier analysis
sentiment_df = pd.DataFrame(results)
print(f"\nProcessed {len(sentiment_df)} headlines")

In [ ]:
# Display results in a table
sentiment_df

## 2.3 Visualize Sentiment Distribution

In [ ]:
# Count sentiment distribution
sentiment_counts = sentiment_df['label'].value_counts()

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of sentiment counts
colors = {'positive': 'green', 'negative': 'red', 'neutral': 'gray'}
bar_colors = [colors.get(label.lower(), 'blue') for label in sentiment_counts.index]

axes[0].bar(sentiment_counts.index, sentiment_counts.values, color=bar_colors, edgecolor='black')
axes[0].set_title('Sentiment Distribution', fontsize=13, pad=15)
axes[0].set_ylabel('Number of Headlines', fontsize=11)
axes[0].set_xlabel('Sentiment', fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

# Confidence score distribution
axes[1].hist(sentiment_df['score'], bins=10, edgecolor='black', alpha=0.7, color='steelblue')
axes[1].set_title('Confidence Score Distribution', fontsize=13, pad=15)
axes[1].set_xlabel('Confidence Score', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].axvline(sentiment_df['score'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {sentiment_df["score"].mean():.3f}')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Sentiment Distribution:")
print(sentiment_counts)

## 2.4 Convert Sentiment to Numerical Score

For quantitative analysis, we need to convert categorical sentiment into numerical values.

**Scoring System:**
- **Positive** → +1 (multiplied by confidence)
- **Neutral** → 0
- **Negative** → -1 (multiplied by confidence)

This creates a score ranging from -1 (very negative) to +1 (very positive).

In [ ]:
def sentiment_to_score(label, confidence):
    """
    Convert sentiment label to numerical score.
    
    Parameters:
    -----------
    label : str
        Sentiment label (positive, negative, neutral)
    confidence : float
        Model confidence score (0-1)
    
    Returns:
    --------
    float : Sentiment score from -1 to +1
    """
    label = label.lower()
    if label == 'positive':
        return confidence
    elif label == 'negative':
        return -confidence
    else:  # neutral
        return 0.0

In [ ]:
# Apply scoring function
sentiment_df['sentiment_score'] = sentiment_df.apply(
    lambda row: sentiment_to_score(row['label'], row['score']), 
    axis=1
)

# Display updated DataFrame
sentiment_df[['headline', 'label', 'score', 'sentiment_score']]

In [ ]:
# Visualize sentiment scores
plt.figure(figsize=(12, 6))

# Create color map based on sentiment
colors_map = sentiment_df['sentiment_score'].apply(
    lambda x: 'green' if x > 0 else ('red' if x < 0 else 'gray')
)

plt.barh(range(len(sentiment_df)), sentiment_df['sentiment_score'], 
         color=colors_map, edgecolor='black', alpha=0.7)
plt.yticks(range(len(sentiment_df)), 
           [h[:50] + '...' if len(h) > 50 else h for h in sentiment_df['headline']], 
           fontsize=9)
plt.xlabel('Sentiment Score', fontsize=11)
plt.title('Financial Headlines - Sentiment Scores', fontsize=13, pad=15)
plt.axvline(0, color='black', linestyle='-', linewidth=1)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print(f"Average Sentiment Score: {sentiment_df['sentiment_score'].mean():.4f}")

---

# PART 3: Building a Cumulative Sentiment Indicator

**Cumulative Sentiment** tracks sentiment over time to identify trends and potential trading signals.

**How it works:**
1. Aggregate daily sentiment scores
2. Calculate cumulative sum over time
3. Identify divergences between sentiment and price
4. Generate trading signals based on sentiment extremes

**Use Cases:**
- **Contrarian signals**: Extreme negative sentiment → potential bottom
- **Confirmation**: Rising prices + rising sentiment → strong trend
- **Divergence**: Price up but sentiment down → warning signal

Let's create a time series of news sentiment and combine it with price data.

## 3.1 Create Synthetic Time Series Data

For demonstration, we'll create a synthetic dataset with dates and headlines. In practice, you would:
- Scrape news from financial APIs (Alpha Vantage, NewsAPI, Bloomberg)
- Use RSS feeds from financial news sites
- Access proprietary news databases

In [ ]:
# Create a time series of financial news
# Simulating 60 days of news (multiple headlines per day)

np.random.seed(42)

# Generate dates
date_range = pd.date_range(start='2024-01-01', end='2024-03-31', freq='D')

# Create diverse headlines with varying sentiment
positive_headlines = [
    "Company reports strong earnings, exceeding expectations",
    "Stock reaches new 52-week high on positive analyst coverage",
    "Firm announces major partnership with tech giant",
    "Revenue growth accelerates in latest quarter",
    "Management raises full-year guidance",
    "New product launch receives enthusiastic market response",
    "Insider buying signals confidence in company outlook",
    "Analyst upgrades stock to buy rating"
]

negative_headlines = [
    "Shares tumble on weak earnings report",
    "Company faces regulatory investigation",
    "Revenue misses estimates, guidance lowered",
    "Key executive announces departure",
    "Product recall announced, costs expected to rise",
    "Analyst downgrades stock to sell on concerns",
    "Market share declines amid increased competition",
    "Quarterly loss wider than expected"
]

neutral_headlines = [
    "Company announces quarterly dividend payment",
    "Firm to report earnings next week",
    "Management to present at investor conference",
    "Trading volume remains steady",
    "Company releases scheduled regulatory filing",
    "Board meeting scheduled for next month"
]

# Generate news data
news_data = []
for date in date_range:
    # Random number of headlines per day (1-4)
    n_headlines = np.random.randint(1, 5)
    
    for _ in range(n_headlines):
        # Random sentiment bias (simulate market regime)
        rand = np.random.random()
        if rand < 0.35:
            headline = np.random.choice(positive_headlines)
        elif rand < 0.70:
            headline = np.random.choice(negative_headlines)
        else:
            headline = np.random.choice(neutral_headlines)
        
        news_data.append({
            'date': date,
            'headline': headline
        })

news_df = pd.DataFrame(news_data)
print(f"Generated {len(news_df)} news headlines over {len(date_range)} days")
print(f"Average {len(news_df)/len(date_range):.1f} headlines per day")
news_df.head(10)

## 3.2 Score All Headlines with FinBERT

Process all headlines through FinBERT to get sentiment scores.

In [ ]:
# Score all headlines (this may take a minute)
print("Scoring headlines with FinBERT...")
print(f"Total headlines to process: {len(news_df)}")

sentiment_results = []
for idx, row in news_df.iterrows():
    result = finbert(row['headline'])[0]
    sentiment_results.append({
        'label': result['label'],
        'score': result['score']
    })

# Add results to dataframe
news_df['sentiment_label'] = [r['label'] for r in sentiment_results]
news_df['sentiment_confidence'] = [r['score'] for r in sentiment_results]
news_df['sentiment_score'] = news_df.apply(
    lambda row: sentiment_to_score(row['sentiment_label'], row['sentiment_confidence']),
    axis=1
)

print(f"\nCompleted! Processed {len(news_df)} headlines")
print(f"\nSample results:")
news_df.head(10)

In [ ]:
# Score all headlines (this may take a minute)
print("Scoring all headlines with FinBERT...")

# Process in batches for efficiency
batch_size = 32
all_sentiments = []

for i in range(0, len(news_df), batch_size):
    batch = news_df.iloc[i:i+batch_size]['headline'].tolist()
    sentiments = finbert(batch)
    all_sentiments.extend(sentiments)

# Add sentiment data to DataFrame
news_df['label'] = [s['label'] for s in all_sentiments]
news_df['confidence'] = [s['score'] for s in all_sentiments]
news_df['sentiment_score'] = news_df.apply(
    lambda row: sentiment_to_score(row['label'], row['confidence']), 
    axis=1
)

print(f"✓ Scored {len(news_df)} headlines")
news_df.head(10)

## 3.3 Aggregate Daily Sentiment

Calculate daily average sentiment scores.

In [ ]:
# Aggregate sentiment by day
daily_sentiment = news_df.groupby('date').agg({
    'sentiment_score': ['mean', 'std', 'count'],
    'headline': 'count'
}).reset_index()

# Flatten column names
daily_sentiment.columns = ['date', 'avg_sentiment', 'sentiment_std', 'count', 'n_headlines']
daily_sentiment = daily_sentiment.set_index('date')

print("Daily Sentiment Statistics:")
print(daily_sentiment.describe())
daily_sentiment.head(10)

## 3.4 Calculate Cumulative Sentiment Indicator

The cumulative sentiment indicator tracks how sentiment accumulates over time.

In [ ]:
# Calculate cumulative sentiment
daily_sentiment['cumulative_sentiment'] = daily_sentiment['avg_sentiment'].cumsum()

# Calculate rolling averages for smoothing
daily_sentiment['sentiment_sma_7'] = daily_sentiment['avg_sentiment'].rolling(window=7).mean()
daily_sentiment['sentiment_sma_20'] = daily_sentiment['avg_sentiment'].rolling(window=20).mean()

print("Cumulative Sentiment Indicator:")
print(f"Starting value: {daily_sentiment['cumulative_sentiment'].iloc[0]:.4f}")
print(f"Ending value:   {daily_sentiment['cumulative_sentiment'].iloc[-1]:.4f}")
print(f"Total change:   {daily_sentiment['cumulative_sentiment'].iloc[-1] - daily_sentiment['cumulative_sentiment'].iloc[0]:.4f}")
daily_sentiment.tail(10)

## 3.5 Visualize Sentiment Over Time

In [ ]:
# Create comprehensive sentiment visualization
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Plot 1: Daily average sentiment with rolling averages
axes[0].bar(daily_sentiment.index, daily_sentiment['avg_sentiment'], 
            color=daily_sentiment['avg_sentiment'].apply(lambda x: 'green' if x > 0 else 'red'),
            alpha=0.3, edgecolor='none', label='Daily Avg')
axes[0].plot(daily_sentiment.index, daily_sentiment['sentiment_sma_7'], 
             color='blue', linewidth=2, label='7-day SMA')
axes[0].plot(daily_sentiment.index, daily_sentiment['sentiment_sma_20'], 
             color='orange', linewidth=2, label='20-day SMA')
axes[0].axhline(0, color='black', linestyle='-', linewidth=0.8)
axes[0].set_ylabel('Daily Sentiment', fontsize=11)
axes[0].set_title('Daily Sentiment with Moving Averages', fontsize=13, pad=15)
axes[0].legend(fontsize=9, loc='upper left')
axes[0].grid(True, alpha=0.3)

# Plot 2: Cumulative sentiment
axes[1].plot(daily_sentiment.index, daily_sentiment['cumulative_sentiment'], 
             color='purple', linewidth=2)
axes[1].fill_between(daily_sentiment.index, 0, daily_sentiment['cumulative_sentiment'],
                     where=(daily_sentiment['cumulative_sentiment'] >= 0), 
                     color='green', alpha=0.2, label='Positive Zone')
axes[1].fill_between(daily_sentiment.index, 0, daily_sentiment['cumulative_sentiment'],
                     where=(daily_sentiment['cumulative_sentiment'] < 0), 
                     color='red', alpha=0.2, label='Negative Zone')
axes[1].axhline(0, color='black', linestyle='-', linewidth=0.8)
axes[1].set_ylabel('Cumulative Sentiment', fontsize=11)
axes[1].set_title('Cumulative Sentiment Indicator', fontsize=13, pad=15)
axes[1].legend(fontsize=9, loc='upper left')
axes[1].grid(True, alpha=0.3)

# Plot 3: News volume (number of headlines per day)
axes[2].bar(daily_sentiment.index, daily_sentiment['n_headlines'], 
            color='steelblue', alpha=0.6, edgecolor='black')
axes[2].set_ylabel('Number of Headlines', fontsize=11)
axes[2].set_xlabel('Date', fontsize=11)
axes[2].set_title('Daily News Volume', fontsize=13, pad=15)
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 3.6 Combine Sentiment with Price Data

Now let's combine sentiment with actual price data to analyze the relationship.

In [ ]:
# Load price data (using the same data source as other notebooks)
price_data = pd.read_csv('http://hilpisch.com/tr_eikon_eod_data.csv', 
                         index_col=0, parse_dates=True)

# Select a stock (Apple)
symbol = 'AAPL.O'
prices = price_data[[symbol]].copy()
prices.columns = ['price']

# Filter to our date range
prices = prices.loc['2024-01-01':'2024-03-31']

print(f"Loaded price data for {symbol}")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")
print(f"Number of trading days: {len(prices)}")
prices.head()

In [ ]:
# Merge sentiment and price data
combined_data = prices.join(daily_sentiment, how='inner')

# Calculate price returns
combined_data['returns'] = combined_data['price'].pct_change()
combined_data['cumulative_returns'] = (1 + combined_data['returns']).cumprod() - 1

# Normalize price for visualization (start at 100)
combined_data['price_normalized'] = combined_data['price'] / combined_data['price'].iloc[0] * 100

print(f"Combined dataset shape: {combined_data.shape}")
combined_data.head()

## 3.7 Visualize Sentiment vs Price

In [ ]:
# Plot sentiment vs price on dual y-axis
fig, ax1 = plt.subplots(figsize=(14, 7))

# Price on left axis
color = 'tab:blue'
ax1.set_xlabel('Date', fontsize=11)
ax1.set_ylabel('Normalized Price (Base=100)', fontsize=11, color=color)
ax1.plot(combined_data.index, combined_data['price_normalized'], 
         color=color, linewidth=2, label='Stock Price')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, alpha=0.3)

# Cumulative sentiment on right axis
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Cumulative Sentiment', fontsize=11, color=color)
ax2.plot(combined_data.index, combined_data['cumulative_sentiment'], 
         color=color, linewidth=2, label='Cumulative Sentiment', alpha=0.7)
ax2.tick_params(axis='y', labelcolor=color)
ax2.axhline(0, color='gray', linestyle='--', alpha=0.5)

plt.title(f'{symbol} Price vs Cumulative Sentiment', fontsize=14, pad=20)

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=10)

plt.tight_layout()
plt.show()

## 3.8 Analyze Sentiment-Return Relationship

Let's analyze if sentiment can predict future returns.

In [ ]:
# Calculate forward returns (next day return)
combined_data['forward_return_1d'] = combined_data['returns'].shift(-1)
combined_data['forward_return_5d'] = combined_data['price'].pct_change(5).shift(-5)

# Correlation analysis
correlation_1d = combined_data[['avg_sentiment', 'forward_return_1d']].corr().iloc[0, 1]
correlation_5d = combined_data[['avg_sentiment', 'forward_return_5d']].corr().iloc[0, 1]

print(f"Sentiment-Return Correlation Analysis:")
print(f"{'='*60}")
print(f"1-day forward return correlation:  {correlation_1d:.4f}")
print(f"5-day forward return correlation:  {correlation_5d:.4f}")
print(f"\nInterpretation:")
if abs(correlation_1d) > 0.3:
    print(f"  Strong correlation between sentiment and next-day returns")
elif abs(correlation_1d) > 0.1:
    print(f"  Moderate correlation between sentiment and next-day returns")
else:
    print(f"  Weak correlation between sentiment and next-day returns")

In [ ]:
# Scatter plot of sentiment vs forward returns
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1-day forward returns
axes[0].scatter(combined_data['avg_sentiment'], combined_data['forward_return_1d'], 
                alpha=0.5, s=50, edgecolor='black')
axes[0].set_xlabel('Daily Sentiment Score', fontsize=11)
axes[0].set_ylabel('Next Day Return', fontsize=11)
axes[0].set_title(f'Sentiment vs 1-Day Forward Return (r={correlation_1d:.3f})', 
                  fontsize=12, pad=15)
axes[0].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[0].axvline(0, color='red', linestyle='--', alpha=0.5)
axes[0].grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(combined_data['avg_sentiment'].dropna(), 
               combined_data['forward_return_1d'].dropna(), 1)
p = np.poly1d(z)
axes[0].plot(combined_data['avg_sentiment'].sort_values(), 
             p(combined_data['avg_sentiment'].sort_values()), 
             "r-", linewidth=2, alpha=0.7, label='Trend')
axes[0].legend(fontsize=9)

# 5-day forward returns
axes[1].scatter(combined_data['avg_sentiment'], combined_data['forward_return_5d'], 
                alpha=0.5, s=50, edgecolor='black', color='orange')
axes[1].set_xlabel('Daily Sentiment Score', fontsize=11)
axes[1].set_ylabel('5-Day Forward Return', fontsize=11)
axes[1].set_title(f'Sentiment vs 5-Day Forward Return (r={correlation_5d:.3f})', 
                  fontsize=12, pad=15)
axes[1].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[1].axvline(0, color='red', linestyle='--', alpha=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3.9 Generate Trading Signals from Sentiment

Create simple trading signals based on sentiment thresholds.

In [ ]:
# Define sentiment-based trading strategy
# Buy when sentiment is positive, sell when negative, hold when neutral

def generate_sentiment_signal(sentiment_score, threshold=0.1):
    """
    Generate trading signal based on sentiment.
    
    Parameters:
    -----------
    sentiment_score : float
        Daily average sentiment score
    threshold : float
        Minimum absolute sentiment to trigger signal
    
    Returns:
    --------
    int : 1 (long), -1 (short), 0 (neutral)
    """
    if sentiment_score > threshold:
        return 1  # Long
    elif sentiment_score < -threshold:
        return -1  # Short
    else:
        return 0  # Neutral

# Apply signal generation
combined_data['signal'] = combined_data['avg_sentiment'].apply(
    lambda x: generate_sentiment_signal(x, threshold=0.1)
)

# Count signals
signal_counts = combined_data['signal'].value_counts()
print(f"Trading Signals Generated:")
print(f"{'='*60}")
print(f"Long signals (1):    {signal_counts.get(1, 0)} days")
print(f"Short signals (-1):  {signal_counts.get(-1, 0)} days")
print(f"Neutral (0):         {signal_counts.get(0, 0)} days")

In [ ]:
# Calculate strategy returns
combined_data['strategy_return'] = combined_data['signal'].shift(1) * combined_data['returns']
combined_data['cumulative_strategy'] = (1 + combined_data['strategy_return'].fillna(0)).cumprod() - 1
combined_data['cumulative_buy_hold'] = combined_data['cumulative_returns']

# Performance metrics
total_strategy_return = combined_data['cumulative_strategy'].iloc[-1]
total_buy_hold_return = combined_data['cumulative_buy_hold'].iloc[-1]

# Sharpe ratio
strategy_sharpe = (combined_data['strategy_return'].mean() / 
                   combined_data['strategy_return'].std() * np.sqrt(252))
buy_hold_sharpe = (combined_data['returns'].mean() / 
                   combined_data['returns'].std() * np.sqrt(252))

print(f"Strategy Performance Comparison:")
print(f"{'='*60}")
print(f"Sentiment Strategy Return:  {total_strategy_return:.2%}")
print(f"Buy & Hold Return:          {total_buy_hold_return:.2%}")
print(f"")
print(f"Sentiment Strategy Sharpe:  {strategy_sharpe:.3f}")
print(f"Buy & Hold Sharpe:          {buy_hold_sharpe:.3f}")

In [ ]:
# Visualize strategy performance
plt.figure(figsize=(14, 7))

plt.plot(combined_data.index, combined_data['cumulative_strategy'] * 100, 
         linewidth=2, label='Sentiment Strategy', color='purple')
plt.plot(combined_data.index, combined_data['cumulative_buy_hold'] * 100, 
         linewidth=2, label='Buy & Hold', color='blue', alpha=0.7)

plt.xlabel('Date', fontsize=11)
plt.ylabel('Cumulative Return (%)', fontsize=11)
plt.title('Sentiment-Based Trading Strategy vs Buy & Hold', fontsize=14, pad=20)
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.axhline(0, color='black', linestyle='-', linewidth=0.8)
plt.tight_layout()
plt.show()

---

## Summary: Key Takeaways

### What We Learned:

**1. HuggingFace Transformers Pipeline:**
- Simple API for using pre-trained NLP models
- Handles tokenization and inference automatically
- Easy to integrate into quantitative workflows
- Supports many NLP tasks beyond sentiment

**2. FinBERT for Financial Sentiment:**
- Domain-specific model trained on financial text
- More accurate than general sentiment models for finance
- Returns 3 classes: Positive, Negative, Neutral
- Confidence scores help assess prediction quality
- Can be converted to numerical scores (-1 to +1)

**3. Cumulative Sentiment Indicator:**
- Aggregates sentiment over time to identify trends
- Rolling averages smooth out daily noise
- Can reveal divergences with price action
- Useful for both trend-following and contrarian strategies
- Volume of news (number of headlines) also matters

**4. Sentiment-Price Relationship:**
- Correlation varies by time horizon
- May be predictive in some market conditions
- Works best combined with other signals
- Consider lag effects (sentiment → price or price → sentiment?)

### Practical Considerations:

**✅ Strengths:**
- Objective, quantifiable measure of market sentiment
- Processes information faster than manual analysis
- Can analyze large volumes of text data
- Captures market psychology and narratives

**⚠️ Limitations:**
- News may lag price action (efficient markets)
- Quality and timeliness of news sources matter
- Model may misinterpret context or sarcasm
- Requires continuous model monitoring and updating
- Overfitting risk in strategy optimization

**💡 Best Practices:**
- Combine with technical and fundamental signals
- Use appropriate sentiment thresholds for your strategy
- Consider sentiment momentum and acceleration
- Account for news source reliability
- Backtest across different market regimes
- Monitor model performance over time

### Next Steps:

To improve this analysis, you could:
1. **Use real-time news APIs** (NewsAPI, Alpha Vantage, Benzinga)
2. **Fine-tune FinBERT** on specific stocks or sectors
3. **Analyze sentiment by source** (Bloomberg vs Twitter vs Reddit)
4. **Calculate sentiment surprises** (actual vs expected sentiment)
5. **Combine with options market sentiment** (put/call ratio)
6. **Build multi-factor models** integrating sentiment with other signals

---

## 📝 Student Assignment

Complete the following tasks to practice sentiment analysis and build your own indicators.

**Expected Time**: 60-90 minutes

---

## Task 1: Advanced Sentiment Scoring

**Objective:** Create alternative sentiment scoring methods and compare their effectiveness.

Implement at least TWO of the following scoring methods:

### Option A: Weighted Sentiment Score

Weight sentiment by confidence and news volume (more headlines = stronger signal).

In [ ]:
# TODO: Implement weighted sentiment scoring
# Hint: Score = sentiment * confidence * sqrt(n_headlines)
# Compare with simple average sentiment

# Your code here

### Option B: Sentiment Momentum

Calculate rate of change in sentiment (sentiment acceleration).

In [ ]:
# TODO: Calculate sentiment momentum
# Hint: momentum = sentiment_today - sentiment_7_days_ago
# Or use percentage change in cumulative sentiment

# Your code here

### Option C: Sentiment Dispersion

Measure disagreement in sentiment (high std = conflicting news).

In [ ]:
# TODO: Analyze sentiment dispersion
# Use sentiment_std from daily aggregation
# High dispersion might signal uncertainty or mixed news

# Your code here

**Analysis Questions for Task 1:**

**TODO: Write your analysis here**

1. Which scoring method had the highest correlation with forward returns?
2. How does sentiment momentum differ from raw sentiment levels?
3. What does high sentiment dispersion tell you about market conditions?
4. Which method would you use in a real trading system and why?

---

## Task 2: Enhanced Trading Strategy

**Objective:** Improve the basic sentiment trading strategy with additional rules.

Implement at least TWO enhancements:

### Enhancement A: Contrarian Strategy

Trade against extreme sentiment (buy panic, sell euphoria).

In [ ]:
# TODO: Implement contrarian sentiment strategy
# When sentiment is extremely negative → Buy
# When sentiment is extremely positive → Sell
# Define "extreme" using z-scores or percentiles

# Your code here

### Enhancement B: Combine Sentiment with Technical Signals

Only trade sentiment signals when confirmed by price action (trend, moving averages, etc.).

In [ ]:
# TODO: Combine sentiment with technical indicators
# Example: Only buy on positive sentiment if price > 20-day MA
# Or require both sentiment AND RSI to agree

# Your code here

### Enhancement C: Dynamic Position Sizing

Vary position size based on sentiment strength and confidence.

In [ ]:
# TODO: Implement dynamic position sizing
# Position size = base_size * abs(sentiment_score) * avg_confidence
# Stronger sentiment → larger position

# Your code here

In [ ]:
# TODO: Compare all strategy variants
# Create performance comparison table with:
# - Total return
# - Sharpe ratio
# - Max drawdown
# - Win rate
# - Number of trades

# Your code here

**Analysis Questions for Task 2:**

**TODO: Write your analysis here**

1. Which strategy enhancement performed best? Why?
2. Does contrarian or momentum approach work better for sentiment?
3. How does combining sentiment with technical signals affect win rate?
4. What are the risks of each approach?

---

## Task 3 (Optional/Advanced): Multi-Stock Sentiment Analysis

**Objective:** Extend sentiment analysis to multiple stocks and create a sentiment-based portfolio.

Build a portfolio that weights stocks by their relative sentiment.

In [ ]:
# TODO: Implement multi-stock sentiment portfolio
# 1. Create separate news/sentiment for multiple stocks (or simulate)
# 2. Calculate sentiment for each stock
# 3. Weight portfolio by relative sentiment (rank-based)
# 4. Rebalance monthly based on sentiment changes
# 5. Compare with equal-weight and market-cap-weighted portfolios

# Your code here

**Analysis for Task 3:**

**TODO: Write your analysis**

1. Does sentiment-based weighting outperform equal-weight?
2. How often does the portfolio rebalance?
3. Which stocks consistently have positive vs negative sentiment?
4. How diversified is the sentiment-based portfolio?

---

## Submission Checklist

Your completed notebook should include:

- [ ] **Task 1 completed** - At least 2 alternative sentiment scoring methods
- [ ] **Task 2 completed** - At least 2 strategy enhancements implemented
- [ ] **At least 3 visualizations** - Sentiment distribution, time series, performance comparison
- [ ] **Performance metrics table** - Comparing all strategies
- [ ] **Written analysis** (4-6 paragraphs) addressing all questions
- [ ] **Code comments** explaining your implementation choices
- [ ] **Conclusion** about sentiment analysis effectiveness

---

## Final Summary & Reflection

**TODO: Write your comprehensive summary here (4-6 paragraphs)**

Your summary should address:

1. **Model Understanding:**
   - How does FinBERT differ from general sentiment models?
   - When would financial-specific models be most valuable?
   - What are the limitations of pre-trained sentiment models?

2. **Sentiment Indicators:**
   - What did cumulative sentiment reveal that daily sentiment didn't?
   - How does sentiment lead, lag, or coincide with price movements?
   - Which sentiment metrics were most predictive?

3. **Trading Strategy Performance:**
   - Did sentiment-based strategies outperform buy-and-hold?
   - What factors contributed to success or failure?
   - How important is the choice of threshold and lookback period?

4. **Practical Application:**
   - What data sources would you use in production?
   - How would you handle real-time sentiment analysis?
   - What additional features would improve the strategy?

5. **Integration with Other Methods:**
   - How could sentiment complement technical analysis?
   - Should sentiment be a primary signal or a filter?
   - When might sentiment be misleading?

6. **Key Learnings:**
   - What surprised you most about sentiment analysis?
   - What would you do differently with more time or data?
   - How does NLP fit into quantitative trading?

---

**Remember:** Sentiment analysis is a powerful tool but works best when:
- Combined with other signals (technical, fundamental)
- Used with appropriate time horizons
- Monitored for model drift and changes in news patterns
- Applied with proper risk management

The future of quantitative trading increasingly involves alternative data sources like sentiment. Understanding how to extract signals from unstructured text data is becoming essential for modern quantitative analysts. 🚀